In [1]:
import time
import socket
from queue import Queue 
from threading import Thread, Condition

In [2]:
from ipywidgets import widgets
from IPython.display import display

class ALE_TextInput:
    
    def __init__(self):
        
        self.Queue_User = Queue()
        self.Text = widgets.Text()
        display(self.Text)
        
        def input_handler(sender):
    
            global thread_running
    
            msg = self.Text.value
            print(msg)
            if msg=='end':
                thread_running = False

            self.Queue_User.put(msg)
#             for i in range (10):
#                 self.Queue_User.put("{}-{}".format(msg, i))
    
        self.Text.on_submit(input_handler)
        
    # to get a message from user queue. this function can block the thread
    def get(self):
        return self.Queue_User.get()

In [3]:
class ALE_TR:
    
    def __init__(self, name, upper_Tx, lower_TR):
        
        self.Name     = name
        self.Upper_Tx = upper_Tx  # must provide get()
        self.Lower_TR = lower_TR  # must provide send(msg), and receive()
        
    def loop_Tx(self):

        global thread_running
        c = 0
    
        while (thread_running == True):

            c = c + 1

            # get text from Upper_Tx, which must provide a get method
            # this thread is blocked here
            msg = self.Upper_Tx.get()
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Tx: message %d: %s"%(self.Name, c, msg))

            # add the text to the queue
            self.Lower_TR.send(msg)
            
    def loop_Rx(self):
        
        global thread_running
        c = 0
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Rx: message %d: %s"%(self.Name, c, msg))

In [4]:
# state of DLE
STATE_READY_TO_SEND = 0 # ready to send a packet to the DLE entity in another node through lower layer
STATE_WAITING_ACK   = 1 # waiting for ACK from the DLE entity in another node

# events of DLE
EVENT_UPPER_TX  = 0 # upper layer wants to send a packet
EVENT_LOWER_DAT = 1 # lower layer forwards an incoming data packet
EVENT_LOWER_ACK = 2 # lower layer forwards an incoming data packet
EVENT_TIMEOUT   = 3 # timer timeout
        
class DLE_TR_FSM:
    
    def __init__(self, name, lower_TR):
        
        self.Name      = name
        self.Lower_TR  = lower_TR   # must provide send(msg) and receive()
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        
        # create a queue for Finite State Machine
        self.Queue_FSM = Queue()
        
        # create a flag = condition to start sending
        self.cv_TxOp = Condition()
        self.TxOp    = True

        # init state to STATE_WAITING_UPPER
        self.State = STATE_READY_TO_SEND
        
        self.Procedure = [[self.FSM_upper_Tx, self.FSM_abnormal],
                          [self.FSM_lower_Rx, self.FSM_lower_Rx],
                          [self.FSM_abnormal, self.FSM_lower_Rx_ack],
                          [self.FSM_abnormal, self.FSM_timeout]]
        
        # timer
        self.cv_Timer      = Condition()
        self.timer_active  = False
        self.timer_counter = 0
        
        # transmission 
        self.tx_buffer  = 0
        self.tx_seq_num = 0
        
        # reception
        self.tx_ack_num = 0
        
        
    def loop_Tx(self):

        global thread_running
        c = 0
    
        while (thread_running == True):

            # waiting for the ready to sent signal
            with self.cv_TxOp: 
                while (self.TxOp != True): 
                    self.cv_TxOp.wait()
            
                self.TxOp = False
                
                c = c + 1

                # get a message from queue
                msg = self.Queue_Tx.get()
                print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
                print("%s Tx: message %d: %s"%(self.Name, c, msg))
            
                self.event_add(EVENT_UPPER_TX, msg) 
            
    def loop_Rx(self):
        
        global thread_running
        c = 0
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Rx: message %d: %s"%(self.Name, c, msg))
            
            # add an event for FSM, assuming that msg is a sequence of bytes
            msg_type = msg[0]
            if (msg_type == 0):
                self.event_add(EVENT_LOWER_DAT, msg[1:])
            elif (msg_type == 1):
                self.event_add(EVENT_LOWER_ACK, msg[1:])
            else:
                print("%s Rx: message type unknown %d"%(self.Name, msg_type))
            
    def receive(self):
        return self.Queue_Rx.get()
    
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def event_add(self, ev_type, msg):
        
        # preparing an event
        if (isinstance(msg, str)):
            msg = msg.encode()
        
        # the event is a sequence of bytes
        event = ev_type.to_bytes(1, "big")+msg
        
        # add event to queue
        self.Queue_FSM.put(event)
        
    def loop_FSM(self):
        
        global thread_running
        
        while (thread_running == True):
            
            # get the next event
            event = self.Queue_FSM.get()
            
            ev_type = event[0]
            
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s FSM: event type: %d"%(self.Name, ev_type))
            
            # process the event
            msg = event[1:]
            self.Procedure[ev_type][self.State](msg)

    def FSM_abnormal(self, msg):
        
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: error! %s"%(self.Name, msg))
    
    def FSM_upper_Tx(self, msg):   
        
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: to send frame %d: %s"%(self.Name, self.tx_seq_num, msg.decode('utf-8')))

        # buffer the message
        self.tx_buffer = msg

        # waiting for ACK
        self.State = STATE_WAITING_ACK
        
        # prepare to send a data message
        msg = bytes([0, self.tx_seq_num]) + self.tx_buffer    # inicating a new packet with sequence number
        self.Lower_TR.send(msg)

        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()       
        
        
    
    def FSM_lower_Rx(self, msg):

        # received a new data packet
        seq = msg[0]              # get the sequence number
        msg = msg[1:].decode()
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: received frame %d: %s"%(self.Name, seq, msg))

        if (seq == self.tx_ack_num):
            
            # put the message in a receiving queue
            self.Queue_Rx.put(msg)
            
            self.tx_ack_num = self.tx_ack_num + 1
            if (self.tx_ack_num == 256):
                self.tx_ack_num = 0
                
        else:
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s FSM: received frame %d but expected %d"%(self.Name, seq, self.tx_ack_num))

        # to send an ACK
        self.Lower_TR.send(bytes([1, seq]))

                
    def FSM_lower_Rx_ack(self, msg):

        # received an ACK                
        
        ack = msg[0] # get the ack number
        if (ack != self.tx_seq_num):
            
            # the received ack does not match the seq
            print("ACK # %d does not match local seq # %d"%(ack, self.tx_seq_num))
            return
        
        # stop timer
        self.timer_active  = False
        
        # inc the seq
        self.tx_seq_num = self.tx_seq_num + 1
        if (self.tx_seq_num == 256):
                self.tx_seq_num = 0
        
        # ready to send the next upper layer packet
        with self.cv_TxOp:
            self.State = STATE_READY_TO_SEND
            self.TxOp  = True
            self.cv_TxOp.notify()
            
    def FSM_timeout(self, msg):
        
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: to resend frame %d: %s"%(self.Name, self.tx_seq_num, self.tx_buffer.decode('utf-8')))

        # prepare to send a data message
        msg = bytes([0, self.tx_seq_num]) + self.tx_buffer    # inicating a new packet with sequence number
        self.Lower_TR.send(msg)

        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()

    def loop_timer(self):
        
        global thread_running
        
        while (thread_running == True):
            
            with self.cv_Timer: 
                while (self.timer_active == False): 
                    self.cv_Timer.wait() 
            
                time.sleep(0.5)

                if (self.timer_counter == 0):
                    # add an event for timeout
                    self.event_add(EVENT_TIMEOUT, bytes([0]))
                    self.timer_active = False

                else:
                    self.timer_counter = self.timer_counter - 1

In [5]:
import random

class PLE_TR:

    def __init__(self, name, Socket, AP_Tx, AP_Rx):
        
        self.Name      = name
        self.Socket    = Socket
        self.AP_Tx     = AP_Tx
        self.AP_Rx     = AP_Rx
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        
    def loop_Tx(self):
        
        global thread_running
        global avg_loss
        global avg_duplication
        
        while (thread_running == True):
        
            # get a message from queue
            msg = self.Queue_Tx.get()
            
            
            time.sleep(random.randint(0,2))
        
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Tx: %s"%(self.Name, msg))
        
            if (random.random() < 0.1): # probability of 10% loss of packet
                print("packet lost")
                avg_loss+=1
            else:
                # sending the message using socket
                if(random.random() < 0.05): # probability of 5% duplication
                    print("duplication ocurred")
                    avg_duplication+=1
                    self.Socket.sendto(msg, self.AP_Tx)
                self.Socket.sendto(msg, self.AP_Tx)

    def loop_Rx(self):
        
        global thread_running
        global bufferSize
    
        # binding the socket with the IP and port
        self.Socket.bind(self.AP_Rx)
        
        while (thread_running == True):
        
            # get a message from socket, this thread is blocked here
            msg_addr = self.Socket.recvfrom(bufferSize)
    
            msg  = msg_addr[0]
            addr = msg_addr[1]
            
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Rx from %s: %s"%(self.Name, addr, msg))        
            self.Queue_Rx.put(msg)
            
    def send(self, msg):
        if (isinstance(msg, str)):
            msg = msg.encode()
        self.Queue_Tx.put(msg)
        
    def receive(self):
        return self.Queue_Rx.get()

In [6]:
class MLE_TR:
    def __init__(self, name, lower_TR, experiment_id=1):
            self.Name      = name
            self.Button    = widgets.Button(description=f'{name}')
            self.out       = widgets.Output()
            self.Lower_TR  = lower_TR   # must provide send(msg) and receive()
            self.exp_id    = experiment_id
            self.count     = 0
            
            # variables being observed
            global epoch
            global avg_delay
            global avg_loss
            global avg_duplication
            def on_button_clicked(_):
              # "linking function with output"
              with self.out:
#                 epoch = time.asctime(time.gmtime(0)) 
              # send 100 mesagess
                for i in range(100):
                    self.Lower_TR.send("{}-{}".format("Test", i))
                    stats = "Exp_ID: {}; msg_id: {}; time: {} ns;".format(self.exp_id, i, time.time_ns())
                    print(stats)
            # linking button and function together using a button's method
            self.Button.on_click(on_button_clicked)
            # displaying button and its output together
            widgets.VBox([self.Button,self.out])
            display(self.Button,self.out) 
            
    def loop_Rx(self):
        global thread_running
        global epoch
        global avg_delay
        global avg_loss
        global avg_duplication
        
        c = 0
    
        while (thread_running == True):
            print("Inside MLE")
            c = c + 1
            self.count += 1
            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Rx: message %d: %s"%(self.Name, c, msg))
            avg_delay+=time.thread_time_ns()
            if self.count != 0:
                print("Avg. Delay {:.2f}; Avg. Loss Rate: {:.2f}; Avg. Duplication {:.2f}".format(avg_delay/self.count,avg_loss/self.count,avg_duplication/self.count))

In [7]:
thread_running = False
bufferSize = 1024

# (1) create physical layer entities
AP_local_1  = ("127.0.0.1", 30000)
AP_remote_1 = ("127.0.0.1", 31111)
Socket_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_1       = PLE_TR("PLE_Alice", Socket_1, AP_remote_1, AP_local_1)

# (2) create date link layer entities
DLE_1 = DLE_TR_FSM("DLE_Alice", PLE_1)

# (3) create application layer entities
# ALE_0 = ALE_TextInput()
# ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1)

In [8]:
# (4) create physical layer entities
AP_local_2  = ("127.0.0.1", 31111)
AP_remote_2 = ("127.0.0.1", 30000)
Socket_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2       = PLE_TR("PLE_Bob", Socket_2, AP_remote_2, AP_local_2)

# (5) create date link layer entities
DLE_2 = DLE_TR_FSM("DLE_Bob", PLE_2)

# (6) create application layer entities
ALE_3 = ALE_TextInput()
ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2)

Text(value='')

In [9]:
avg_delay = 0
avg_loss = 0
avg_duplication = 0
epoch = None
# (7) create physical layer entities to test 
AP_local_T_1  = ("127.0.0.1", 30030)
AP_remote_T_1 = ("127.0.0.1", 31120)
Socket_T_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_T_1       = PLE_TR("PLE_T_Alice", Socket_T_1, AP_remote_T_1, AP_local_T_1)

# DLE_T_1 = DLE_TR_FSM("DLE_T_Alice", PLE_T_1)


AP_local_T_2  = ("127.0.0.1", 31120)
AP_remote_T_2 = ("127.0.0.1", 30030)
Socket_T_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_T_2       = PLE_TR("PLE_T_Bob", Socket_T_2, AP_remote_T_2, AP_local_T_2)

DLE_T_2 = DLE_TR_FSM("DLE_T_Bob", PLE_T_2)


# ALE_T_0 = ALE_TextInput()
# ALE_T_2 = ALE_TR("ALE_T_Bob", ALE_T_0, DLE_T_2)


In [10]:
# (7.5) Measure PLE_TR object
MLE_1 = MLE_TR("Measure PLE_T_1", PLE_T_1, 1)

Button(description='Measure PLE_T_1', style=ButtonStyle())

Output()

In [11]:
MLE_2 = MLE_TR("Measure DLE_T_1", DLE_T_2, 2)

Button(description='Measure DLE_T_1', style=ButtonStyle())

Output()

In [12]:
# start the loops of all entities
# all loops must be blocked at a certain position

# t1_1 = Thread(target = ALE_1.loop_Tx, args = ()) 
# t2_1 = Thread(target = ALE_1.loop_Rx, args = ()) 
t3_1 = Thread(target = DLE_1.loop_Tx, args = ())
t4_1 = Thread(target = DLE_1.loop_Rx, args = ())
t5_1 = Thread(target = PLE_1.loop_Tx, args = ()) 
t6_1 = Thread(target = PLE_1.loop_Rx, args = ())
f1_1 = Thread(target = DLE_1.loop_FSM, args = ())
f2_1 = Thread(target = DLE_1.loop_timer, args = ())

t1_2 = Thread(target = ALE_2.loop_Tx, args = ()) 
t2_2 = Thread(target = ALE_2.loop_Rx, args = ()) 
t3_2 = Thread(target = DLE_2.loop_Tx, args = ())
t4_2 = Thread(target = DLE_2.loop_Rx, args = ())
t5_2 = Thread(target = PLE_2.loop_Tx, args = ()) 
t6_2 = Thread(target = PLE_2.loop_Rx, args = ()) 
f1_2 = Thread(target = DLE_2.loop_FSM, args = ())
f2_2 = Thread(target = DLE_2.loop_timer, args = ())


# '''
# '''
m1_t_1 = Thread(target = MLE_1.loop_Rx, args = ()) 
# t3_t_1 = Thread(target = DLE_T_1.loop_Tx, args = ())
# t4_t_1 = Thread(target = DLE_T_1.loop_Rx, args = ())
t5_t_1 = Thread(target = PLE_T_1.loop_Tx, args = ()) 
t6_t_1 = Thread(target = PLE_T_1.loop_Rx, args = ())

m1_t_2 = Thread(target = MLE_2.loop_Rx, args = ()) 
# t1_t_2 = Thread(target = ALE_T_2.loop_Tx, args = ()) 
# t2_t_2 = Thread(target = ALE_T_2.loop_Rx, args = ()) 
t3_t_2 = Thread(target = DLE_T_2.loop_Tx, args = ())
t4_t_2 = Thread(target = DLE_T_2.loop_Rx, args = ())
t5_t_2 = Thread(target = PLE_T_2.loop_Tx, args = ()) 
t6_t_2 = Thread(target = PLE_T_2.loop_Rx, args = ())

thread_running = True

# t1_1.start()
# t2_1.start()
t3_1.start()
t4_1.start()
t5_1.start()
t6_1.start()
t1_2.start()
t2_2.start()
t3_2.start()
t4_2.start()
t5_2.start()
t6_2.start()
f1_1.start()
f1_2.start()
f2_1.start()
f2_2.start()

''''''
m1_t_1.start()
# t3_t_1.start()
# t4_t_1.start()
t5_t_1.start()
t6_t_1.start()

m1_t_2.start()
# t1_t_2.start()
# t2_t_2.start()
t3_t_2.start()
t4_t_2.start()
t5_t_2.start()
t6_t_2.start()

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\users\aics1\appdata\local\programs\python\python37-32\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "c:\users\aics1\appdata\local\programs\python\python37-32\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-2877324f9e43>", line 48, in loop_Rx
    self.Socket.bind(self.AP_Rx)
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

Exception in thread Thread-17:
Traceback (most recent call last):
  File "c:\users\aics1\appdata\local\programs\python\python37-32\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "c:\users\aics1\appdata\local\programs\python\python37-32\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-2877324f9e43>", line 48, in loop_Rx
    self.Socket.bind(self.AP_Rx)
OSError: [

Inside MLE
Inside MLE


In [13]:
# print(t1_1.is_alive())
# print(t2_1.is_alive())
print(t3_1.is_alive())
print(t4_1.is_alive())
print(t5_1.is_alive())
print(t6_1.is_alive())

print(t1_2.is_alive())
print(t2_2.is_alive())
print(t3_2.is_alive())
print(t4_2.is_alive())
print(t5_2.is_alive())
print(t6_2.is_alive())

print(f1_1.is_alive())
print(f1_2.is_alive())
print(f2_1.is_alive())
print(f2_2.is_alive())

print(m1_t_1.is_alive())
# print(t3_t_1.is_alive())
# print(t4_t_1.is_alive())
print(t5_t_1.is_alive())
print(t6_t_1.is_alive())

print(m1_t_2.is_alive())
# print(t1_t_2.is_alive())
# print(t2_t_2.is_alive())
print(t3_t_2.is_alive())
print(t4_t_2.is_alive())
print(t5_t_2.is_alive())
print(t6_t_2.is_alive())



True
True
True
False
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
